In [1]:
!pip install funpymodeling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=45bda8061ece1f5ae6c8945d67ce95b55c8d0edea791560cc42da849326f9eaf
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2


In [2]:
#Importacion de las librerias
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from funpymodeling.exploratory import cat_vars, num_vars
import numpy as np

### Preparacion de Datos

In [3]:
# Carga de datos
data=pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv")

# Removemos duplicados de canciones:
data=data.drop_duplicates(subset="track_id")

#Nos quedamos unicamente con las variables numericas
x_data=data.drop(cat_vars(data), axis=1)

# Sacamos algunas variables adicionales que no aportan valor
x_data=x_data.drop(['key','speechiness', 'mode', 'tempo', 'duration_ms'], axis=1)

### Creación del modelo de PCA

In [4]:
#Importamos la libreria para el escalado de los datos
from sklearn.preprocessing import StandardScaler

#Generamos el objeto
scaler = StandardScaler()

#Aplicamos la transformacion
x_scaled = scaler.fit_transform(x_data)

#Importante: Los datos no tienen que tener nulos y deben ser todos numericos

Generamos el modelo y fiteamos:

In [5]:
#Importamos la libreria
from sklearn.decomposition import PCA

#Generamos el objeto
model_pca = PCA()

#Aplicamos pca
x_pca=model_pca.fit_transform(x_scaled)

In [6]:
#Variaanza explicada de las componentes
var_explicada_pca = model_pca.explained_variance_ratio_
var_explicada_pca

array([0.26718277, 0.17844569, 0.14004096, 0.12199857, 0.10753268,
       0.08223481, 0.0748838 , 0.02768073])

**Interpretación**:

La primer componente aporta el 26 % de la varianza explicada, la segunda el 17% y así sucesivamente

# UMAP

UMAP es un método de reducción de dimensionalidad no lineal y es muy eficaz para visualizar agrupaciones o grupos de puntos de datos y sus proximidades relativas.

Link de Interes: 

* https://towardsdatascience.com/dimensionality-reduction-for-data-visualization-pca-vs-tsne-vs-umap-be4aa7b1cb29

In [7]:
!pip3 install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=9f88df115598eca28d83ffc1036619bc92c95ef66aeb1e95b2f00ff580209b22
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=070a395a07f9f6f212f485e4201795ee807f8f5054faed319b7b9d633f6a3114
  Stored in directory: /root/.cache/pip/wheels/1c/63/3a/29954bca1a27ba100ed8c27973a78cb71b43dc67aed62e80c3
Successfully built umap-learn pynndescent


In [8]:
import umap #pip3 install umap-learn

In [9]:
#Generamos el objeto para la estandarizacion
x_scaled = StandardScaler()

#Aplicamos la estandarizacion
x_scaled = x_scaled.fit_transform(x_data)

In [10]:
#Obtenemos el objeto umap
model_umap = umap.UMAP()

In [11]:
#Ejecutamos el umap
model_umap_fit_transform = model_umap.fit_transform(x_scaled)
model_umap_fit_transform.shape

(28356, 2)

Visualización interactiva con Plotly!

In [12]:
data2 = data.copy() #Hacemos un copy
data2['cancion'] = data2['track_artist'] + ' | ' + data2['track_name'] #Creamos una nueva variable llamada: cancion
data2[['dim1', 'dim2']] = model_umap_fit_transform #Agregamos las dos dimensiones generadas por umap

In [13]:
import plotly.express as px

fig = px.scatter(data2, x="dim1", y="dim2", color="track_popularity", hover_data=['cancion'])
fig.show()